In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys
from joblib import Parallel, delayed

key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

In [ ]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
train_test = pd.concat([df_train, df_test], axis=0)
# sys.exit()
df_hist = utils.read_df_pkl('../input/hist_clean_rdm0*')
# df_hist = utils.read_df_pkl('../input/hist_exist*')
df_hist = reduce_mem_usage(df_hist)
# df_new = utils.read_df_pkl('../input/new_mer*0*')
df_new = utils.read_df_pkl('../input/new_clean_rdm0*')
# df_new = utils.read_df_pkl('../input/new_exist*')
df_new = reduce_mem_usage(df_new)
df_trans = pd.concat([df_hist, df_new], axis=0)

df_org_lag = utils.read_pkl_gzip('../input/train_test_original_month_lag.gz')
df_org_lag = reduce_mem_usage(df_org_lag)
# df_org_lag.head()

df_org_lag.rename(columns={'purchase_date':'yyyymm'}, inplace=True)
print(df_trans.shape)
df_trans = df_trans.merge(df_org_lag.reset_index()[[key, 'yyyymm', 'org_month_lag']], how='inner', on=[key, 'yyyymm'])
print(df_trans.shape)

#========================================================================
# Category
#========================================================================
df_trans = df_trans[df_trans['category_1']==1]
df = df_trans
auth1 = df[df.authorized_flag==1]
auth0 = df[df.authorized_flag==0]

In [ ]:
historical_transactions_df = historical_transactions_df.sort_values(by=['card_id', 'purchase_date'])[['card_id', 'month_lag', 'purchase_amount']]
card_ids = list(historical_transactions_df['card_id'].unique())
out_dfs = [None] * len(card_ids)
a = 0
agg_func = {'purchase_amount': ['count','mean', 'sum', 'median', 'nunique'],}
historical_transactions_df_by_month_lag = historical_transactions_df.groupby(['card_id', 'month_lag']).agg(agg_func)
historical_transactions_df_by_month_lag.columns = ['_'.join(map(str,col)) for col in historical_transactions_df_by_month_lag.columns]
regr = LinearRegression()
col_names = ['card_id', 'coef', 'intercept']
df = pd.DataFrame(columns=col_names)
for card_id in card_ids:
  x = list(historical_transactions_df_by_month_lag.loc[card_id].index)
  y = list(historical_transactions_df_by_month_lag.loc[card_id]['purchase_amount_sum'])
  x = [[x] for x in x]
  y = [[x] for x in y]
  reg = regr.fit(x, y)
  coef = (reg.coef_[0][0])
  intercept = (reg.intercept_[0])
  out_dfs[a] = {'card_id': card_id, 'coef': coef, 'intercept': intercept}
  a += 1
for col in df.columns:
  if col != 'card_id':
    features.append(col)
df_regression = pd.concat([pd.DataFrame([out_dfs[i]], columns=col_names) for i in range(len(list(filter(None, out_dfs))))],ignore_index=True)